In [13]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

In [14]:
model_data = pd.read_pickle("data_for_model.pkl")
# model_data

Build a regression model.

In [21]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

# Load your data (already done)
# model_data = pd.read_pickle("data_for_model.pkl")

# Reduce the number of unique POI categories
top_n = 10
category_counts = model_data['poi_category'].value_counts()
top_categories = category_counts.index[:top_n]
model_data['poi_category_reduced'] = model_data['poi_category'].apply(
    lambda x: x if x in top_categories else 'Other'
)

# One-hot encode the reduced poi_category column
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = encoder.fit_transform(model_data[['poi_category_reduced']])
encoded_df = pd.DataFrame(
    encoded, 
    columns=encoder.get_feature_names_out(['poi_category_reduced']),
    index=model_data.index
)


# Prepare model dataset
df_model = model_data[['free_bikes', 'poi_distance', 'empty_slots', 'num_pois']].dropna()
df_model = pd.concat([df_model, encoded_df], axis=1)

# Define X and y
X = df_model.drop(columns=['free_bikes'])
y = df_model['free_bikes']
X = sm.add_constant(X)

# Fit model
model = sm.OLS(y, X).fit()

Provide model output and an interpretation of the results. 

In [23]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     1205.
Date:                Mon, 28 Jul 2025   Prob (F-statistic):               0.00
Time:                        16:42:20   Log-Likelihood:            -1.3392e+05
No. Observations:               39808   AIC:                         2.679e+05
Df Residuals:                   39794   BIC:                         2.680e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

1. R-Squared: The model explains 28.2% of the variance in free_bikes. This is a modest but not unusual result for behavioral or observational data in public infrastructure.
2. P-values (P>|t|): If p-value < 0.05, the variable is statistically significant (it has a real impact).
   * poi_distance: Coef = -0.0009, p < 0.001 (statistically significant) - For every additional meter between the station and its POIs, the number of free bikes decreases slightly. Possibly because more centrally located stations (closer to POIs) are more frequently emptied by users.
   * empty_slots: Coef = -0.4628, p < 0.001 (statistically significant), More empty slots are strongly associated with fewer free bikes, which makes sense operationally.
   * num_pois - Not statistically significant (p = 0.673): Despite being intuitive, the raw count of POIs does not seem to explain variation in free bikes once other factors are included.
   * Categorical Variables:
     * Most categories (e.g., Pub, Restaurant, Pizzeria, etc.) have negative coefficients, suggesting that their presence is associated with fewer free bikes.
     * French Restaurant has one of the largest negative effects (-4.01 bikes) — possibly because stations near them are popular or in high-demand areas.
     * All the POI categories are statistically significant (p < 0.001), meaning the differences are unlikely to be due to chance.
     